# Excel Quality Pipeline – Discovery Notebook

## Objective
Build a reproducible data pipeline that produces a single cleaned
quality dataset from multiple Excel inputs.

This notebook is for exploration and validation only.
Final logic will be migrated into the `src/` module.


In [1]:
import pandas as pd
import numpy as np
pd.__version__


'2.3.3'

In [2]:
# ===== Monthly Run Config =====
OPD_FILE = "../data/raw/feb/PATIENT OPD VISITS-2026_2_1_10_24.xlsx"
LIVER_FILE = "../data/raw/feb/Liver Registry - 1 Feb.xlsx"
KIDNEY_RECIP_FILE = "../data/raw/feb/Kidney Recipient - 1 Feb.xlsx"
KIDNEY_DONOR_FILE = "../data/raw/feb/Kidney Donor - 1 Feb.xlsx"
MOTC_FILE = "../data/raw/feb/MOTC OPD visits - from 2018-2021.xlsx"

OUTPUT_FILE = "../data/output/cleaned_opd_visits.xlsx"
# =============================


In [3]:
df = pd.read_excel(OPD_FILE)
df.shape

(3810, 18)

In [4]:
header = df.columns.tolist()

In [5]:
print(header)

['MRN', 'PatEngName', 'pat_birthdate', 'regtime', 'VISIT_DATETIME', 'ARRIVETIME', 'STARTTIME', 'VISIT_TYPE', 'PORTAL STATUS', 'VISIT STATUS', 'BOOKING_TYPE', 'clinic_key', 'CLINIC_NAME', 'MRP', 'JOB TITLE', 'SPECIALTY', 'Added_by', 'addtime']


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3810 entries, 0 to 3809
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   MRN             3810 non-null   int64         
 1   PatEngName      3749 non-null   object        
 2   pat_birthdate   3749 non-null   datetime64[ns]
 3   regtime         3749 non-null   datetime64[ns]
 4   VISIT_DATETIME  3810 non-null   object        
 5   ARRIVETIME      3087 non-null   datetime64[ns]
 6   STARTTIME       2990 non-null   datetime64[ns]
 7   VISIT_TYPE      3810 non-null   object        
 8   PORTAL STATUS   3749 non-null   object        
 9   VISIT STATUS    3749 non-null   object        
 10  BOOKING_TYPE    3749 non-null   object        
 11  clinic_key      3749 non-null   float64       
 12  CLINIC_NAME     3810 non-null   object        
 13  MRP             3810 non-null   object        
 14  JOB TITLE       3749 non-null   object        
 15  SPEC

In [7]:
# Drop uneeded col
cols_to_drop = [
    "regtime",
    "ARRIVETIME",
    "STARTTIME",
    "PORTAL STATUS",
    "clinic_key",
    "JOB TITLE",
    "SPECIALTY",
    "Added_by",
    "addtime",
]

df = df.drop(columns=cols_to_drop)


In [8]:
df.columns

Index(['MRN', 'PatEngName', 'pat_birthdate', 'VISIT_DATETIME', 'VISIT_TYPE',
       'VISIT STATUS', 'BOOKING_TYPE', 'CLINIC_NAME', 'MRP'],
      dtype='object')

In [9]:
# Normalize column names
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)
df.columns

Index(['mrn', 'patengname', 'pat_birthdate', 'visit_datetime', 'visit_type',
       'visit_status', 'booking_type', 'clinic_name', 'mrp'],
      dtype='object')

In [10]:
df['visit_datetime'].dtype

dtype('O')

In [11]:
# change col visit_datetime to datetime DD/MM/YYYY

df["visit_datetime"] = pd.to_datetime(
    df["visit_datetime"],
    errors="coerce"
).dt.date


In [12]:
df["visit_datetime"].dtype
type(df.loc[0, "visit_datetime"])


datetime.date

In [13]:
# Adding the new col

new_cols = [
    "liver_r_d",
    "liver_status",
    "kidney_recipient",
    "kidney_donor",
]

for col in new_cols:
    df[col] = None

df.columns

Index(['mrn', 'patengname', 'pat_birthdate', 'visit_datetime', 'visit_type',
       'visit_status', 'booking_type', 'clinic_name', 'mrp', 'liver_r_d',
       'liver_status', 'kidney_recipient', 'kidney_donor'],
      dtype='object')

In [14]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor
0,220920,WARDA MOHAMMED ALI AL HAMAD,1977-09-30,2026-01-12,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,None,None,None,None
1,187647,ALREEM AHMED ABDULKAREEM KHALAF,2016-08-31,2026-01-06,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,Mohammed Abdulmoatani Almaghrabi,None,None,None,None
2,218085,Sultan Ali Saleh Alnahdi,1990-07-01,2026-01-13,NEW,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,ABDULNASER MOHAMMED HAMOUD ALABADI,None,None,None,None
3,220888,NAWAF ABDULLAH EISSA ALSHAMMARI,2019-11-12,2026-01-04,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,None,None,None,None
4,147118,SAAD ABDULLAH ALI ALBUAINAIN,2008-12-31,2026-01-25,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,None,None,None,None


In [15]:
df['visit_status'].value_counts()

visit_status
STARTED    2990
NO SHOW     662
ARRIVED      97
Name: count, dtype: int64

In [16]:
# Remove NO SHOW visit_status

df = df[df["visit_status"] != "NO SHOW"]


In [17]:
df['visit_status'].value_counts()

visit_status
STARTED    2990
ARRIVED      97
Name: count, dtype: int64

In [18]:
# Fill liver columns using Liver Registry (ONLY)
# For each MRN in df:
# Look it up in Liver Registry
# If found:
###### Rec_Don → liver_r_d
###### Current Status1 → liver_status
# If not found:
##### leave as NaN

liver_df = pd.read_excel(LIVER_FILE)
liver_df.columns = (liver_df.columns.str.strip().str.lower().str.replace(" ", "_"))



In [19]:
liver_df.columns.tolist()



['r_mrn',
 'id_number',
 'patient_name',
 'mobile1_0',
 'payment_0',
 'rec_don',
 'nationality',
 'city_of_residence',
 'specify_other',
 'age_group',
 'age',
 'blood_group',
 'gender',
 'date_open_file_0',
 'current_status1',
 'patient_type_1',
 'initial_tx_assessment_date',
 'workup_start_date',
 'workup_completed_date',
 'date_added_to_waiting_list',
 'surgery_date',
 'tx_type',
 'transplant_place',
 'comment_0',
 'closefile_r',
 'closefiledate_r',
 'closefilereason_r',
 'assigned_consultant_0',
 'assigned_surgeon_r',
 'primary_disease']

In [20]:
df = df.merge(
    liver_df[["r_mrn", "rec_don", "current_status1"]],
    left_on="mrn",
    right_on="r_mrn",
    how="left"
)


In [21]:
df["liver_r_d"] = df["rec_don"]
df["liver_status"] = df["current_status1"]
df.tail()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,r_mrn,rec_don,current_status1
3143,222221,NaN,NaT,2026-01-11,new,NaN,NaN,Psychology,Feda Al Ghamdi,NaN,NaN,None,None,NaN,NaN,NaN
3144,177889,NaN,NaT,2026-01-12,new,NaN,NaN,Psychology,Feda Al Ghamdi,Recipient,New patient,None,None,177889,Recipient,New patient
3145,222472,NaN,NaT,2026-01-19,new,NaN,NaN,Psychology,Feda Al Ghamdi,Donor,Pre Tx on workups,None,None,222472,Donor,Pre Tx on workups
3146,191339,NaN,NaT,2026-01-19,new,NaN,NaN,Psychology,Feda Al Ghamdi,NaN,NaN,None,None,NaN,NaN,NaN
3147,222858,NaN,NaT,2026-01-25,new,NaN,NaN,Psychology,Feda Al Ghamdi,Donor,Pre Tx on workups,None,None,222858,Donor,Pre Tx on workups


In [22]:
df = df.drop(columns=["r_mrn", "rec_don", "current_status1"])


In [23]:
df[df["liver_r_d"].notna()]


,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor
4,147118,SAAD ABDULLAH ALI ALBUAINAIN,2008-12-31,2026-01-25,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,Recipient,Hepatology patient,None,None
88,112285,ALI MOHAMMED ALI ALSHARDAN,2017-04-13,2026-01-18,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Marwah Abdullah Al-Mubarak,Recipient,Post Tx,None,None
90,137912,SAAD HAMAD HASSAN SAHHARI,2018-12-05,2026-01-14,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Marwah Abdullah Al-Mubarak,Recipient,Hepatology patient,None,None
93,120160,NAJD TALAL HABIB ALANAZI,2017-05-24,2026-01-14,VV,STARTED,NORMAL BOOKING,Clinical Nutrition,Marwah Abdullah Al-Mubarak,Recipient,Hepatology patient,None,None
95,119981,HASHIM ABDULRAHEEM NASSER AL HAMAD,2017-12-02,2026-01-14,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Marwah Abdullah Al-Mubarak,Recipient,Post Tx,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,127093,NaN,NaT,2026-01-05,Follow up,NaN,NaN,Psychology,Alia Al Enazi,Recipient,Pre Tx on DDWL,None,None
3140,210060,NaN,NaT,2026-01-06,new,NaN,NaN,Psychology,Alia Al Enazi,Recipient,Hepatology patient,None,None
3144,177889,NaN,NaT,2026-01-12,new,NaN,NaN,Psychology,Feda Al Ghamdi,Recipient,New patient,None,None
3145,222472,NaN,NaT,2026-01-19,new,NaN,NaN,Psychology,Feda Al Ghamdi,Donor,Pre Tx on workups,None,None


In [24]:
# Fill kidney_recipient using Kidney Recipient registry (ONLY)
# For each mrn in the main df:
# Look it up in Kidney Recipient - 1 JAN.xlsx

# If found:
###### copy Current Status1 → kidney_recipient
# If not found:
###### leave as NaN

kidney_rec_df = pd.read_excel(KIDNEY_RECIP_FILE)
kidney_rec_df.columns = (kidney_rec_df.columns.str.strip().str.lower().str.replace(" ", "_"))


In [25]:
df = df.merge(
    kidney_rec_df[["r_mrn", "current_status1"]],
    left_on="mrn",
    right_on="r_mrn",
    how="left"
)
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,r_mrn,current_status1
0,220920,WARDA MOHAMMED ALI AL HAMAD,1977-09-30,2026-01-12,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,None,None,NaN,NaN
1,187647,ALREEM AHMED ABDULKAREEM KHALAF,2016-08-31,2026-01-06,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,Mohammed Abdulmoatani Almaghrabi,NaN,NaN,None,None,187647.0,Post LD Tx
2,218085,Sultan Ali Saleh Alnahdi,1990-07-01,2026-01-13,NEW,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,ABDULNASER MOHAMMED HAMOUD ALABADI,NaN,NaN,None,None,NaN,NaN
3,220888,NAWAF ABDULLAH EISSA ALSHAMMARI,2019-11-12,2026-01-04,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,None,None,NaN,NaN
4,147118,SAAD ABDULLAH ALI ALBUAINAIN,2008-12-31,2026-01-25,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,Recipient,Hepatology patient,None,None,147118.0,Post DD Tx


In [26]:
df["kidney_recipient"] = df["current_status1"]
df = df.drop(columns=["r_mrn", "current_status1"])



In [27]:
df.tail()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor
3143,222221,NaN,NaT,2026-01-11,new,NaN,NaN,Psychology,Feda Al Ghamdi,NaN,NaN,NaN,None
3144,177889,NaN,NaT,2026-01-12,new,NaN,NaN,Psychology,Feda Al Ghamdi,Recipient,New patient,Pre Tx on hold,None
3145,222472,NaN,NaT,2026-01-19,new,NaN,NaN,Psychology,Feda Al Ghamdi,Donor,Pre Tx on workups,NaN,None
3146,191339,NaN,NaT,2026-01-19,new,NaN,NaN,Psychology,Feda Al Ghamdi,NaN,NaN,NaN,None
3147,222858,NaN,NaT,2026-01-25,new,NaN,NaN,Psychology,Feda Al Ghamdi,Donor,Pre Tx on workups,NaN,None


In [28]:
# Fill kidney_donor
# For each mrn in the main df:
# Match it to d_mrn in Kidney Donor - 1 JAN.xlsx
# Copy current_status1_d → kidney_donor
# If no match → leave NaN

kidney_donor_df = pd.read_excel(KIDNEY_DONOR_FILE)

kidney_donor_df.columns = (
    kidney_donor_df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
)

df = df.merge(
    kidney_donor_df[["d_mrn", "current_status1_d"]],
    left_on="mrn",
    right_on="d_mrn",
    how="left"
)

df["kidney_donor"] = df["current_status1_d"]

df = df.drop(columns=["d_mrn", "current_status1_d"])


In [29]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor
0,220920,WARDA MOHAMMED ALI AL HAMAD,1977-09-30,2026-01-12,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,NaN,NaN
1,187647,ALREEM AHMED ABDULKAREEM KHALAF,2016-08-31,2026-01-06,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,Mohammed Abdulmoatani Almaghrabi,NaN,NaN,Post LD Tx,NaN
2,218085,Sultan Ali Saleh Alnahdi,1990-07-01,2026-01-13,NEW,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,ABDULNASER MOHAMMED HAMOUD ALABADI,NaN,NaN,NaN,Post living donor nephrectomy
3,220888,NAWAF ABDULLAH EISSA ALSHAMMARI,2019-11-12,2026-01-04,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,NaN,NaN
4,147118,SAAD ABDULLAH ALI ALBUAINAIN,2008-12-31,2026-01-25,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,Recipient,Hepatology patient,Post DD Tx,NaN


In [30]:
df['clinic_name'].value_counts()

clinic_name
Anesthesia                                         760
Clinical Nutrition                                 602
Adult Post Renal Transplant                        469
TRANSPLANT SURGERY                                 410
Adult Post Kidney Transplant Virtual Clinic        228
ADULT POST LIVER TRANSPLANT                        128
Adult Pre Renal Transplant                          96
Adult Pre Liver Transplant                          88
Pediatric Post Renal Transplant                     85
Pediatric Liver Transplant                          72
Social Work                                         46
Adult Post Liver Transplant Virtual                 40
Transplant Dressing (procedure)                     38
Pediatric Post Kidney Transplant Virtual Clinic     21
Adult Pre Renal Transplant Virtual Clinic           19
Psychology                                          15
Pediatric Pre Renal Transplant                      11
Adult Pre Liver Transplant Virtual                  1

In [31]:
# Remove visits based on clinic + transplant status (ONLY)

# Remove row if ALL of the following are true:

# 1-clinic_name is Anesthesia OR Clinical Nutrition
# 2- liver_r_d is NaN
# 3-kidney_recipient is NaN
# 4-kidney_donor is NaN

mask = (
    df["clinic_name"].isin(["Anesthesia", "Clinical Nutrition"])
    & df["liver_r_d"].isna()
    & df["kidney_recipient"].isna()
    & df["kidney_donor"].isna()
)

df = df[~mask]


In [32]:
df[
    df["clinic_name"].isin(["Anesthesia", "Clinical Nutrition"])
][["liver_r_d", "kidney_recipient", "kidney_donor"]].isna().all(axis=1).sum()


np.int64(0)

In [33]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor
1,187647,ALREEM AHMED ABDULKAREEM KHALAF,2016-08-31,2026-01-06,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,Mohammed Abdulmoatani Almaghrabi,NaN,NaN,Post LD Tx,NaN
2,218085,Sultan Ali Saleh Alnahdi,1990-07-01,2026-01-13,NEW,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,ABDULNASER MOHAMMED HAMOUD ALABADI,NaN,NaN,NaN,Post living donor nephrectomy
4,147118,SAAD ABDULLAH ALI ALBUAINAIN,2008-12-31,2026-01-25,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,Recipient,Hepatology patient,Post DD Tx,NaN
6,210109,YASAMIYAN HAMAD MOHAMMED ALRAWILI,2008-05-02,2026-01-20,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN
28,176279,RADHI NASER HASAN ALHODAR,1946-05-31,2026-01-04,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Ebtisam Hassan Husain Al-Ghawi,NaN,NaN,Nephrology patient,NaN


In [34]:
df[df[["liver_r_d", "kidney_recipient", "kidney_donor"]].isna().all(axis=1)]

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor
1228,213897,RAZAN SAEED JAMAN ALZAHRANI,1999-08-19,2026-01-14,Walk-In,ARRIVED,WALK-IN,Adult Post Renal Transplant,MAHER MOHAMMED RAFAT ALDEMERDASH,NaN,NaN,NaN,NaN
1351,223022,BADER ABDULMOHSEN MOHAMMED ALMUTIRI,1966-10-15,2026-01-28,NEW,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,HADI MOHAMMED SHOEI KURIRY,NaN,NaN,NaN,NaN
2123,209901,Jiji THOMAS,1991-05-05,2026-01-13,Kidney,STARTED,NORMAL BOOKING,TRANSPLANT SURGERY,Mohammed Saad Ali AL-Qahtani,NaN,NaN,NaN,NaN
2167,208567,GHAITH GHALI DAKHEL ALJAMEELI,2017-12-20,2026-01-18,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Liver Transplant,Razan Monther Abdulmajied Bader,NaN,NaN,NaN,NaN
2273,13459,SHAIKHAH MOHAMMED A ALSAMHAN,1940-08-04,2026-01-19,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,Eyad Mohammad Osman Elhassan Gadour,NaN,NaN,NaN,NaN
2293,69960,AZIZAH MOHAMMED A ALSHAHRANI,1979-11-26,2026-01-20,Liver,STARTED,NORMAL BOOKING,TRANSPLANT SURGERY,Mohammed Saad Ali AL-Qahtani,NaN,NaN,NaN,NaN
2595,186450,SAUD EID RASHED AL RASHIDI,2019-10-21,2026-01-14,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Liver Transplant,IBRAHIM ELHADI IBRAHIM HASSAN,NaN,NaN,NaN,NaN
2643,211782,HALIMA IBRAHIM SALEH ALMAJDOUIE,1948-05-10,2026-01-29,Walk-In,ARRIVED,WALK-IN,TRANSPLANT SURGERY,Mohammed Saad Ali AL-Qahtani,NaN,NaN,NaN,NaN
2699,8909,BANEEN ABDULLAH MOHAMMED ALSAIHATI,1986-05-17,2026-01-19,NEW,STARTED,NORMAL BOOKING,TRANSPLANT SURGERY,MAHMOUD ALI MAHMOUD OBEID,NaN,NaN,NaN,NaN
2800,4592,MUNEERAH ALI HAMAD ALSAFRANI,1956-02-12,2026-01-29,Walk-In,ARRIVED,WALK-IN,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,NaN,NaN


In [35]:
# Add need_review flag
df["need_review"] = "no"
review_mask = (
    df[["liver_r_d", "kidney_recipient", "kidney_donor"]]
    .isna()
    .all(axis=1)
)

df.loc[review_mask, "need_review"] = "yes"


In [36]:
df['liver_r_d'].value_counts()


liver_r_d
Recipient    490
Donor         50
Name: count, dtype: int64

In [37]:
# Create and fill r_d (LOCKED LOGIC)

# We look at three columns:
#liver_r_d, kidney_recipient, kidney_donor
# A) If only one of the three is non-NaN:
# Only liver_r_d filled → r_d = "Liver " + liver_r_d
# Only kidney_recipient filled → r_d = kidney_recipient
# Only kidney_donor filled → r_d = kidney_donor
# B) If liver_r_d is non-NaN AND (either kidney_recipient OR kidney_donor is non-NaN):
# Use clinic_name keyword:
# If clinic_name contains renal or kidney → r_d becomes the non-NaN kidney value (recipient preferred, else donor)
# If clinic_name contains liver → r_d = "Liver " + liver_r_d

# Vectorized version
mask_liver_only = df['liver_r_d'].notna() & df['kidney_recipient'].isna() & df['kidney_donor'].isna()
mask_k_recip_only = df['kidney_recipient'].notna() & df['liver_r_d'].isna() & df['kidney_donor'].isna()
mask_k_donor_only = df['kidney_donor'].notna() & df['liver_r_d'].isna() & df['kidney_recipient'].isna()

df['r_d'] = np.select(
    [
        mask_liver_only,
        mask_k_recip_only,
        mask_k_donor_only,
    ],
    [
        "Liver " + df['liver_r_d'].astype(str),
        df['kidney_recipient'],
        df['kidney_donor']
    ],
    default=np.nan
)

# Now handle mixed cases (where liver + at least one kidney is present)
mixed_mask = (
    df['liver_r_d'].notna() &
    (df['kidney_recipient'].notna() | df['kidney_donor'].notna()) &
    (df['r_d'].isna())  # still null after previous step
)

clinic_lower = df['clinic_name'].fillna('').str.lower()

renal_clinic = clinic_lower.str.contains('renal|kidney')
liver_clinic = clinic_lower.str.contains('liver')

df.loc[mixed_mask & renal_clinic & ~liver_clinic, 'r_d'] = \
    df.loc[mixed_mask & renal_clinic & ~liver_clinic, 'kidney_recipient'].fillna(
        df.loc[mixed_mask & renal_clinic & ~liver_clinic, 'kidney_donor']
    )

df.loc[mixed_mask & liver_clinic & ~renal_clinic, 'r_d'] = \
    "Liver " + df.loc[mixed_mask & liver_clinic & ~renal_clinic, 'liver_r_d'].astype(str)




In [38]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d
1,187647,ALREEM AHMED ABDULKAREEM KHALAF,2016-08-31,2026-01-06,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,Mohammed Abdulmoatani Almaghrabi,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx
2,218085,Sultan Ali Saleh Alnahdi,1990-07-01,2026-01-13,NEW,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,ABDULNASER MOHAMMED HAMOUD ALABADI,NaN,NaN,NaN,Post living donor nephrectomy,no,Post living donor nephrectomy
4,147118,SAAD ABDULLAH ALI ALBUAINAIN,2008-12-31,2026-01-25,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,Recipient,Hepatology patient,Post DD Tx,NaN,no,Post DD Tx
6,210109,YASAMIYAN HAMAD MOHAMMED ALRAWILI,2008-05-02,2026-01-20,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx
28,176279,RADHI NASER HASAN ALHODAR,1946-05-31,2026-01-04,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Ebtisam Hassan Husain Al-Ghawi,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient


In [39]:
df['liver_status'].value_counts()

liver_status
Post Tx                        318
Hepatology patient              91
New patient                     37
Pre Tx on workups               37
Pre Tx on DDWL                  26
OPD Hepatology patient          17
Pre Tx ready for transplant      6
Pre Tx Ready for transplant      4
Hepatitis C patient              3
Pre Tx on hold                   1
Name: count, dtype: int64

In [40]:
# ambiguous_mask = (
#     df['liver_r_d'].notna() &
#     (df['kidney_recipient'].notna() | df['kidney_donor'].notna()) &
#     df['r_d'].isna()
# )

# len(df.loc[ambiguous_mask])

# come to this later at the end to see how many should be assigned as need review
df.head(30)

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d
1,187647,ALREEM AHMED ABDULKAREEM KHALAF,2016-08-31,2026-01-06,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,Mohammed Abdulmoatani Almaghrabi,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx
2,218085,Sultan Ali Saleh Alnahdi,1990-07-01,2026-01-13,NEW,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,ABDULNASER MOHAMMED HAMOUD ALABADI,NaN,NaN,NaN,Post living donor nephrectomy,no,Post living donor nephrectomy
4,147118,SAAD ABDULLAH ALI ALBUAINAIN,2008-12-31,2026-01-25,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,Recipient,Hepatology patient,Post DD Tx,NaN,no,Post DD Tx
6,210109,YASAMIYAN HAMAD MOHAMMED ALRAWILI,2008-05-02,2026-01-20,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx
28,176279,RADHI NASER HASAN ALHODAR,1946-05-31,2026-01-04,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Ebtisam Hassan Husain Al-Ghawi,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient
61,201290,AYAT ALI ABDULLAH ASQIT,1994-02-21,2026-01-04,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,FAIY JALAWI AMER AL MUTAIRI,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx
68,218481,AMAL ALI HUSAIN AL ABBAS,1973-11-21,2026-01-28,VV,STARTED,NORMAL BOOKING,Clinical Nutrition,Ebtisam Hassan Husain Al-Ghawi,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient
79,192067,WAEL SALEH BIN ALI ALMUNTASHIRI,2011-07-25,2026-01-28,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Marwah Abdullah Al-Mubarak,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx
83,128975,RAWAN MOSSA AHMAD ALESSA,2014-03-04,2026-01-11,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Marwah Abdullah Al-Mubarak,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient
85,185562,RAED IBRAHIM ABDELMOHSEN ALABDULALI,2013-07-16,2026-01-28,VV,STARTED,NORMAL BOOKING,Clinical Nutrition,Marwah Abdullah Al-Mubarak,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx


In [41]:
df['r_d'].value_counts()

r_d
Post LD Tx                                        855
Liver Recipient                                   413
Pre Tx on workups                                 190
Post living donor nephrectomy                     140
Post DD Tx                                         85
Liver Donor                                        48
Pre Tx Ready for committee                         38
Pre Tx Discussed & Ready for LD Tx                 26
Pre Tx Discussed & ready for donor nephrectomy     23
Waiting for first appointment                      23
Nephrology patient                                 20
Listed on DDWL                                     20
Pre Tx on hold                                     14
Initial visit done, No LD, suspended for DDWL       9
Pre Tx Discussed with pending issues                7
Listed on DDWL - Inactive                           3
Name: count, dtype: int64

In [42]:
df["patient_type"] = None
df["transplant_date"] = None
df["days"] = None


In [43]:
df.head(1)

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days
1,187647,ALREEM AHMED ABDULKAREEM KHALAF,2016-08-31,2026-01-06,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,Mohammed Abdulmoatani Almaghrabi,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,None,None,None


In [44]:
df['patient_type'] = df['liver_status'].combine_first(
    df['kidney_recipient']
).combine_first(
    df['kidney_donor']
)

In [45]:
# if patient_type is NA i think need_revuew = yes

In [46]:
# Create lookup dictionaries: MRN → Surgery Date
# Using the actual column names from your files

liver_dict = (
    liver_df[['r_mrn', 'surgery_date']]
    .dropna(subset=['r_mrn'])
    .set_index('r_mrn')['surgery_date']
    .to_dict()
)

recip_dict = (
    kidney_rec_df[['r_mrn', 'surgery_date']]           # ← using kidney_rec_df (your naming)
    .dropna(subset=['r_mrn'])
    .set_index('r_mrn')['surgery_date']
    .to_dict()
)

donor_dict = (
    kidney_donor_df[['d_mrn', 'surgery_date_d']]
    .dropna(subset=['d_mrn'])
    .set_index('d_mrn')['surgery_date_d']
    .to_dict()
)

# Populate transplant_date with this priority order:
# 1. Liver
# 2. Kidney Recipient
# 3. Kidney Donor
df['transplant_date'] = (
    df['mrn'].map(liver_dict)                  # Liver first
    .combine_first(df['mrn'].map(recip_dict))       # then Recipient
    .combine_first(df['mrn'].map(donor_dict))       # then Donor
)

# Optional: Flag rows that still have no transplant date
# (uncomment when you're ready to implement review flagging)
# df.loc[df['transplant_date'].isna(), 'need_review'] = 'yes'

print("transplant_date column updated.")
print(f"Number of rows with transplant_date filled: {df['transplant_date'].notna().sum()}")

transplant_date column updated.
Number of rows with transplant_date filled: 1422


In [47]:
# fill patient_type: if visit_date - transplant_date is minus, zero or the transplant_date is NaN -> pre
# if the visit_date - transplant_date. patient_type -> post

# Ensure both columns are datetime (safe, coercive)
visit_dt = pd.to_datetime(df["visit_datetime"], errors="coerce")
tx_dt = pd.to_datetime(df["transplant_date"], errors="coerce")

# Calculate day difference
df["days"] = (visit_dt - tx_dt).dt.days

# Overwrite patient_type based on rules
df.loc[tx_dt.isna(), "patient_type"] = "pre"
df.loc[tx_dt.notna() & (df["days"] <= 0), "patient_type"] = "pre"
df.loc[tx_dt.notna() & (df["days"] > 0), "patient_type"] = "post"


In [48]:
df['r_d'].value_counts()

r_d
Post LD Tx                                        855
Liver Recipient                                   413
Pre Tx on workups                                 190
Post living donor nephrectomy                     140
Post DD Tx                                         85
Liver Donor                                        48
Pre Tx Ready for committee                         38
Pre Tx Discussed & Ready for LD Tx                 26
Pre Tx Discussed & ready for donor nephrectomy     23
Waiting for first appointment                      23
Nephrology patient                                 20
Listed on DDWL                                     20
Pre Tx on hold                                     14
Initial visit done, No LD, suspended for DDWL       9
Pre Tx Discussed with pending issues                7
Listed on DDWL - Inactive                           3
Name: count, dtype: int64

In [49]:
df[df['r_d'] == 'Nephrology patient']

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days
28,176279,RADHI NASER HASAN ALHODAR,1946-05-31,2026-01-04,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Ebtisam Hassan Husain Al-Ghawi,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
68,218481,AMAL ALI HUSAIN AL ABBAS,1973-11-21,2026-01-28,VV,STARTED,NORMAL BOOKING,Clinical Nutrition,Ebtisam Hassan Husain Al-Ghawi,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
83,128975,RAWAN MOSSA AHMAD ALESSA,2014-03-04,2026-01-11,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Marwah Abdullah Al-Mubarak,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
323,42618,RANA ABDULAZIZ SAAD AL HAWAS,1972-06-20,2026-01-05,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Entisar Malik M. AL-Hany,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
401,201585,SUJA KAITHAVANA,1973-05-23,2026-01-19,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
453,176618,RAHMAH ABDULRAZAQ ABDULHADI AL HAMMAD,1984-09-10,2026-01-12,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Fatima Abdullah Ali Al-Sinan,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
548,69833,KHADIJAH IBRAHIM HASSAN AL QATTAN,1954-03-06,2026-01-25,NEW,STARTED,NORMAL BOOKING,Anesthesia,MOHAMED ESSAMELDIN HASSAN ABDELMEGUID,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
720,198355,NAWAL ALI A ALGHANNAM,1963-11-17,2026-01-14,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Fatimah Abdullah AlIbrahim,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
725,140801,FAWZYAH ANBAR MOBARAK ALABDULLAH,1956-02-12,2026-01-19,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Fatimah Abdullah AlIbrahim,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN
732,19202,MUSTAFA NASSER HUSSAIN ALJAFAR,1967-10-04,2026-01-05,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Fatimah Abdullah AlIbrahim,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,pre,NaT,NaN


In [50]:
df['liver_status'].value_counts()

liver_status
Post Tx                        318
Hepatology patient              91
New patient                     37
Pre Tx on workups               37
Pre Tx on DDWL                  26
OPD Hepatology patient          17
Pre Tx ready for transplant      6
Pre Tx Ready for transplant      4
Hepatitis C patient              3
Pre Tx on hold                   1
Name: count, dtype: int64

In [51]:
df['patient_type'].value_counts()

patient_type
post    1350
pre      614
Name: count, dtype: int64

In [52]:
# Final patient_type classification logic
#
# This step overwrites patient_type (which initially contains only "pre" / "post")
# and assigns a final clinical category using transplant context and liver status.
#
# Priority order (top to bottom – first match wins):
#
# 1) Hepatology override:
#    If liver_status contains any of:
#       - "hepatology"
#       - "hepatitis"
#       - "new patients"
#    → patient_type = "Hepatology"
#
# 2) Liver transplant context:
#    If r_d contains "liver":
#       - patient_type == "post" → "Post-Liver Tx"
#       - patient_type == "pre"  → "Pre-Liver Tx"
#
# 3) Nephrology (non-transplant):
#    If r_d contains "nephrology":
#       → patient_type = "Nephrology"
#
# 4) Kidney transplant context:
#    If r_d is present AND does NOT contain "liver" or "nephrology":
#       - patient_type == "post" → "Post-Kidney Tx"
#       - patient_type == "pre"  → "Pre-Kidney Tx"
#
# 5) Final fallback:
#    Any remaining rows → "Pre-Kidney Tx"
#
# Notes:
# - Matching is case-insensitive
# - Rules are mutually exclusive and applied in strict priority order
# - This mirrors the quality department’s manual classification logic



import numpy as np


# Prepare case-insensitive conditions
r_d_lower = df['r_d'].fillna('').str.lower()
liver_status_lower = df['liver_status'].fillna('').str.lower()

# ── Priority conditions (applied top → bottom) ───────────────────────────────

# Priority 1: Hepatology override (absolute)
is_hepatology = liver_status_lower.str.contains('hepatology|hepatitis|new patients', na=False)

# Priority 2: Liver transplant context
is_liver_context = r_d_lower.str.contains('liver', na=False)
is_post = df['patient_type'] == 'post'
is_pre  = df['patient_type'] == 'pre'

# Priority 3: Nephrology (non-transplant)
is_nephrology = r_d_lower.str.contains('nephrology', na=False)

# Priority 4: Kidney transplant context
# (has r_d value, but not liver & not nephrology)
is_kidney_context = (
    df['r_d'].notna() &
    ~is_liver_context &
    ~is_nephrology
)

# ── Apply rules in strict priority order using np.select ─────────────────────

conditions = [
    is_hepatology,                                      # 1. Hepatology override
    is_liver_context & is_post,                         # 2. Post-Liver Tx
    is_liver_context & is_pre,                          # 2. Pre-Liver Tx
    is_nephrology,                                      # 3. Nephrology
    is_kidney_context & is_post,                        # 4. Post-Kidney Tx
    is_kidney_context & is_pre,                         # 4. Pre-Kidney Tx
]

choices = [
    "Hepatology",
    "Post-Liver Tx",
    "Pre-Liver Tx",
    "Nephrology",
    "Post-Kidney Tx",
    "Pre-Kidney Tx",
]

df['patient_type'] = np.select(conditions, choices, default="Pre-Kidney Tx")



In [53]:
df['patient_type'].value_counts()

patient_type
Post-Kidney Tx    994
Pre-Kidney Tx     444
Post-Liver Tx     308
Hepatology        111
Pre-Liver Tx       87
Nephrology         20
Name: count, dtype: int64

In [54]:
df['clinic_name'].value_counts()

clinic_name
Adult Post Renal Transplant                        469
TRANSPLANT SURGERY                                 410
Adult Post Kidney Transplant Virtual Clinic        228
ADULT POST LIVER TRANSPLANT                        128
Anesthesia                                         102
Adult Pre Renal Transplant                          96
Adult Pre Liver Transplant                          88
Pediatric Post Renal Transplant                     85
Clinical Nutrition                                  76
Pediatric Liver Transplant                          72
Social Work                                         46
Adult Post Liver Transplant Virtual                 40
Transplant Dressing (procedure)                     38
Pediatric Post Kidney Transplant Virtual Clinic     21
Adult Pre Renal Transplant Virtual Clinic           19
Psychology                                          15
Pediatric Pre Renal Transplant                      11
Adult Pre Liver Transplant Virtual                  1

In [55]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days
1,187647,ALREEM AHMED ABDULKAREEM KHALAF,2016-08-31,2026-01-06,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,Mohammed Abdulmoatani Almaghrabi,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2024-01-23,714.0
2,218085,Sultan Ali Saleh Alnahdi,1990-07-01,2026-01-13,NEW,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,ABDULNASER MOHAMMED HAMOUD ALABADI,NaN,NaN,NaN,Post living donor nephrectomy,no,Post living donor nephrectomy,Post-Kidney Tx,2025-12-02,42.0
4,147118,SAAD ABDULLAH ALI ALBUAINAIN,2008-12-31,2026-01-25,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,Recipient,Hepatology patient,Post DD Tx,NaN,no,Post DD Tx,Hepatology,2024-01-22,734.0
6,210109,YASAMIYAN HAMAD MOHAMMED ALRAWILI,2008-05-02,2026-01-20,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2025-09-15,127.0
28,176279,RADHI NASER HASAN ALHODAR,1946-05-31,2026-01-04,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Ebtisam Hassan Husain Al-Ghawi,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,Nephrology,NaT,NaN


In [56]:
# Create lookup dictionaries: MRN → Age Group

liver_age_dict = (
    liver_df[['r_mrn', 'age_group']]
    .dropna(subset=['r_mrn'])
    .set_index('r_mrn')['age_group']
    .to_dict()
)

recip_age_dict = (
    kidney_rec_df[['r_mrn', 'age_group']]
    .dropna(subset=['r_mrn'])
    .set_index('r_mrn')['age_group']
    .to_dict()
)

donor_age_dict = (
    kidney_donor_df[['d_mrn', 'age_group_d']]
    .dropna(subset=['d_mrn'])
    .set_index('d_mrn')['age_group_d']
    .to_dict()
)

# Populate age_group with priority: Liver → Recipient → Donor
df['age_group'] = (
    df['mrn'].map(liver_age_dict)                # 1. Liver first
    .combine_first(df['mrn'].map(recip_age_dict))     # 2. Kidney Recipient
    .combine_first(df['mrn'].map(donor_age_dict))     # 3. Kidney Donor
)



In [57]:
df['age_group'].value_counts()

age_group
Adult        1686
Pediatric     260
Name: count, dtype: int64

In [58]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days,age_group
1,187647,ALREEM AHMED ABDULKAREEM KHALAF,2016-08-31,2026-01-06,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,Mohammed Abdulmoatani Almaghrabi,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2024-01-23,714.0,Pediatric
2,218085,Sultan Ali Saleh Alnahdi,1990-07-01,2026-01-13,NEW,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,ABDULNASER MOHAMMED HAMOUD ALABADI,NaN,NaN,NaN,Post living donor nephrectomy,no,Post living donor nephrectomy,Post-Kidney Tx,2025-12-02,42.0,Adult
4,147118,SAAD ABDULLAH ALI ALBUAINAIN,2008-12-31,2026-01-25,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,Recipient,Hepatology patient,Post DD Tx,NaN,no,Post DD Tx,Hepatology,2024-01-22,734.0,Pediatric
6,210109,YASAMIYAN HAMAD MOHAMMED ALRAWILI,2008-05-02,2026-01-20,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2025-09-15,127.0,Adult
28,176279,RADHI NASER HASAN ALHODAR,1946-05-31,2026-01-04,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Ebtisam Hassan Husain Al-Ghawi,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,Nephrology,NaT,NaN,Adult


In [59]:
df['clinic_name'].value_counts()

clinic_name
Adult Post Renal Transplant                        469
TRANSPLANT SURGERY                                 410
Adult Post Kidney Transplant Virtual Clinic        228
ADULT POST LIVER TRANSPLANT                        128
Anesthesia                                         102
Adult Pre Renal Transplant                          96
Adult Pre Liver Transplant                          88
Pediatric Post Renal Transplant                     85
Clinical Nutrition                                  76
Pediatric Liver Transplant                          72
Social Work                                         46
Adult Post Liver Transplant Virtual                 40
Transplant Dressing (procedure)                     38
Pediatric Post Kidney Transplant Virtual Clinic     21
Adult Pre Renal Transplant Virtual Clinic           19
Psychology                                          15
Pediatric Pre Renal Transplant                      11
Adult Pre Liver Transplant Virtual                  1

In [60]:
df['patient_type'].value_counts()

patient_type
Post-Kidney Tx    994
Pre-Kidney Tx     444
Post-Liver Tx     308
Hepatology        111
Pre-Liver Tx       87
Nephrology         20
Name: count, dtype: int64

In [61]:
df['r_d'].value_counts()

r_d
Post LD Tx                                        855
Liver Recipient                                   413
Pre Tx on workups                                 190
Post living donor nephrectomy                     140
Post DD Tx                                         85
Liver Donor                                        48
Pre Tx Ready for committee                         38
Pre Tx Discussed & Ready for LD Tx                 26
Pre Tx Discussed & ready for donor nephrectomy     23
Waiting for first appointment                      23
Nephrology patient                                 20
Listed on DDWL                                     20
Pre Tx on hold                                     14
Initial visit done, No LD, suspended for DDWL       9
Pre Tx Discussed with pending issues                7
Listed on DDWL - Inactive                           3
Name: count, dtype: int64

In [62]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days,age_group
1,187647,ALREEM AHMED ABDULKAREEM KHALAF,2016-08-31,2026-01-06,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,Mohammed Abdulmoatani Almaghrabi,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2024-01-23,714.0,Pediatric
2,218085,Sultan Ali Saleh Alnahdi,1990-07-01,2026-01-13,NEW,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,ABDULNASER MOHAMMED HAMOUD ALABADI,NaN,NaN,NaN,Post living donor nephrectomy,no,Post living donor nephrectomy,Post-Kidney Tx,2025-12-02,42.0,Adult
4,147118,SAAD ABDULLAH ALI ALBUAINAIN,2008-12-31,2026-01-25,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,Recipient,Hepatology patient,Post DD Tx,NaN,no,Post DD Tx,Hepatology,2024-01-22,734.0,Pediatric
6,210109,YASAMIYAN HAMAD MOHAMMED ALRAWILI,2008-05-02,2026-01-20,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2025-09-15,127.0,Adult
28,176279,RADHI NASER HASAN ALHODAR,1946-05-31,2026-01-04,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Ebtisam Hassan Husain Al-Ghawi,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,Nephrology,NaT,NaN,Adult


In [63]:
df['r_d'].value_counts()

r_d
Post LD Tx                                        855
Liver Recipient                                   413
Pre Tx on workups                                 190
Post living donor nephrectomy                     140
Post DD Tx                                         85
Liver Donor                                        48
Pre Tx Ready for committee                         38
Pre Tx Discussed & Ready for LD Tx                 26
Pre Tx Discussed & ready for donor nephrectomy     23
Waiting for first appointment                      23
Nephrology patient                                 20
Listed on DDWL                                     20
Pre Tx on hold                                     14
Initial visit done, No LD, suspended for DDWL       9
Pre Tx Discussed with pending issues                7
Listed on DDWL - Inactive                           3
Name: count, dtype: int64

In [64]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days,age_group
1,187647,ALREEM AHMED ABDULKAREEM KHALAF,2016-08-31,2026-01-06,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,Mohammed Abdulmoatani Almaghrabi,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2024-01-23,714.0,Pediatric
2,218085,Sultan Ali Saleh Alnahdi,1990-07-01,2026-01-13,NEW,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,ABDULNASER MOHAMMED HAMOUD ALABADI,NaN,NaN,NaN,Post living donor nephrectomy,no,Post living donor nephrectomy,Post-Kidney Tx,2025-12-02,42.0,Adult
4,147118,SAAD ABDULLAH ALI ALBUAINAIN,2008-12-31,2026-01-25,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,Recipient,Hepatology patient,Post DD Tx,NaN,no,Post DD Tx,Hepatology,2024-01-22,734.0,Pediatric
6,210109,YASAMIYAN HAMAD MOHAMMED ALRAWILI,2008-05-02,2026-01-20,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Post LD Tx,Post-Kidney Tx,2025-09-15,127.0,Adult
28,176279,RADHI NASER HASAN ALHODAR,1946-05-31,2026-01-04,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Ebtisam Hassan Husain Al-Ghawi,NaN,NaN,Nephrology patient,NaN,no,Nephrology patient,Nephrology,NaT,NaN,Adult


In [65]:
df[df['patient_type'] == 'Hepatology'][30:50]

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days,age_group
1287,220675,SALIHAH SAEED ABDULRAHMAN ALGHAMDI,1959-01-10,2026-01-18,V Follow UP,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant Virtual,HADI MOHAMMED SHOEI KURIRY,Recipient,Hepatology patient,NaN,NaN,no,Liver Recipient,Hepatology,NaT,NaN,Adult
1288,160545,Jana EID FAHAAD ALHAJRI,2008-07-05,2026-01-11,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,HADI MOHAMMED SHOEI KURIRY,Recipient,Hepatology patient,NaN,NaN,no,Liver Recipient,Hepatology,NaT,NaN,Adult
1294,217189,SALAF DOKHI SALEM ALSHAMMARI,1977-07-21,2026-01-19,CHART CHECK,STARTED,WALK-IN,Adult Pre Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,Recipient,Hepatology patient,Post LD Tx,NaN,no,Post LD Tx,Hepatology,2026-01-20,-1.0,Adult
1312,200089,HASIRAH MOHAMMED SUDDIQ SAYED QAYSI,1992-09-08,2026-01-20,V Follow UP,ARRIVED,NORMAL BOOKING,Adult Pre Liver Transplant Virtual,NOORA HASSAN B AL FARAJ,Recipient,Hepatology patient,NaN,NaN,no,Liver Recipient,Hepatology,NaT,NaN,Adult
1316,1470,KHALED SAEED BIN AWADH ALQAHTANI,1986-04-26,2026-01-27,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,NOORA HASSAN B AL FARAJ,Recipient,Hepatology patient,NaN,NaN,no,Liver Recipient,Hepatology,NaT,NaN,Adult
1342,196718,MESHAL FAHAD SARI ALHAJRI,2007-02-04,2026-01-27,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,NOORA HASSAN B AL FARAJ,Recipient,Hepatology patient,NaN,NaN,no,Liver Recipient,Hepatology,NaT,NaN,Adult
1343,119922,KHALID AHMED ATIAH ALREESH,1978-06-06,2026-01-27,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,NOORA HASSAN B AL FARAJ,Recipient,OPD Hepatology patient,NaN,NaN,no,Liver Recipient,Hepatology,NaT,NaN,Adult
1344,118047,RAAD MAJED GHALLAB ALHARBI,2017-06-30,2026-01-08,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,Mohammed Abdulmoatani Almaghrabi,Recipient,Hepatology patient,Post LD Tx,NaN,no,Post LD Tx,Hepatology,2019-12-11,2220.0,Pediatric
1350,151311,AWATIF AHMED ALI ALMUBARK,1963-11-17,2026-01-25,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,HADI MOHAMMED SHOEI KURIRY,Recipient,OPD Hepatology patient,NaN,NaN,no,Liver Recipient,Hepatology,NaT,NaN,Adult
1354,76667,LATEEFAH R F ALANEZI,1985-03-11,2026-01-11,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,HADI MOHAMMED SHOEI KURIRY,Recipient,Hepatology patient,NaN,NaN,no,Liver Recipient,Hepatology,NaT,NaN,Adult


In [66]:

# Final logic for correcting / filling r_d column
# Rules applied in this strict top-to-bottom priority order:

# 1. If patient_type == "Hepatology"
#    → r_d = "Liver Recipient"

# 2. If patient_type == "Nephrology"
#    → r_d = "Kidney Recipient"

# 3. If patient_type in ["Post-Liver Tx", "Pre-Liver Tx"]
#    → keep the current r_d value as-is 
#      (it should already be either "Liver Recipient" or "Liver Donor")

# 4. If patient_type in ["Post-Kidney Tx", "Pre-Kidney Tx"]
#    → Decision order:
#      - If kidney_recipient is NOT NULL → r_d = "Kidney Recipient"
#      - Else if kidney_donor is NOT NULL   → r_d = "Kidney Donor"
#      - Else (both kidney_recipient AND kidney_donor are NULL)
#        → keep current r_d value unchanged
#        → AND set need_review = "yes" (change from default "no")

# All other cases: keep current r_d (though in practice should be covered above)

# Notes:
# - Liver cases trust existing r_d (already cleaned correctly)
# - Kidney cases prefer recipient status when available
# - Only flag for manual review when there's true ambiguity 
#   (no clear kidney role in source registry columns)


# 1. Hepatology & Nephrology overrides
hepatology_mask = df['patient_type'] == "Hepatology"
nephrology_mask = df['patient_type'] == "Nephrology"

df.loc[hepatology_mask, 'r_d'] = "Liver Recipient"
df.loc[nephrology_mask, 'r_d'] = "Kidney Recipient"

# 2. Liver cases - keep current r_d (no change needed)

# 3. Kidney transplant cases (Post & Pre)
kidney_tx_mask = df['patient_type'].isin(["Post-Kidney Tx", "Pre-Kidney Tx"])

# Sub-conditions for kidney cases
has_recipient = df['kidney_recipient'].notna()
has_donor     = df['kidney_donor'].notna()

# Apply preference: Recipient > Donor > keep current
df.loc[kidney_tx_mask & has_recipient, 'r_d'] = "Kidney Recipient"
df.loc[kidney_tx_mask & ~has_recipient & has_donor, 'r_d'] = "Kidney Donor"

# 4. Ambiguous kidney cases → flag for review
ambiguous_kidney = (
    kidney_tx_mask & 
    ~has_recipient & 
    ~has_donor
)

df.loc[ambiguous_kidney, 'need_review'] = "yes"


In [67]:
df['clinic_name'].value_counts()

clinic_name
Adult Post Renal Transplant                        469
TRANSPLANT SURGERY                                 410
Adult Post Kidney Transplant Virtual Clinic        228
ADULT POST LIVER TRANSPLANT                        128
Anesthesia                                         102
Adult Pre Renal Transplant                          96
Adult Pre Liver Transplant                          88
Pediatric Post Renal Transplant                     85
Clinical Nutrition                                  76
Pediatric Liver Transplant                          72
Social Work                                         46
Adult Post Liver Transplant Virtual                 40
Transplant Dressing (procedure)                     38
Pediatric Post Kidney Transplant Virtual Clinic     21
Adult Pre Renal Transplant Virtual Clinic           19
Psychology                                          15
Pediatric Pre Renal Transplant                      11
Adult Pre Liver Transplant Virtual                  1

In [68]:
df['r_d'].value_counts()

r_d
Kidney Recipient    1128
Liver Recipient      458
Kidney Donor         312
Liver Donor           48
Name: count, dtype: int64

In [69]:
# Logic for creating the new 'clinic' column
# Rules are applied in order - first matching condition wins (top-to-bottom priority)

# All string matching is:
# - case-insensitive
# - partial contains (anywhere in the clinic_name string)

# 1. If clinic_name contains "renal" or "kidney"
#    → clinic = "Kidney Tx"

# 2. Else if clinic_name contains any of:
#    - "pediatric liver transplant"
#    - "adult pre liver transplant"
#    - "adult post liver transplant"
#    → clinic = "liver Tx"

# 3. Else if clinic_name contains "anesthesia"
#    → Check r_d (case-insensitive contains):
#      - if r_d contains "kidney" → clinic = "Anesthesia-Kidney"
#      - if r_d contains "liver"  → clinic = "Anesthesia-Liver"
#      - if neither → clinic remains NaN

# 4. Else if clinic_name contains "transplant urology" or "transplant cystoscopy clinic"
#    → clinic = "Urology"

# 5. Else if clinic_name contains "clinical nutrition"
#    → clinic = "Nutrition"

# 6. Else if clinic_name contains "transplant dressing (procedure)"
#    → clinic = "Surgery-Dressing"

# 7. Else if clinic_name contains "transplant surgery"
#    → Check r_d (case-insensitive contains):
#      - if r_d contains "kidney" → clinic = "Surgery-Kidney"
#      - else → clinic = "Surgery-Liver"

# 8. Else if clinic_name contains "psychology"
#    → clinic = "Psychology"

# 9. Else if clinic_name contains "social work"
#    → clinic = "Social Work"

# 10. Fallback (no match above)
#     → clinic remains NaN



# Make sure clinic_name is string and handle NaN
df['clinic_name'] = df['clinic_name'].astype(str).fillna('')

# Prepare lowercase versions for matching
clinic_lower = df['clinic_name'].str.lower()
rd_lower     = df['r_d'].fillna('').str.lower()

# Initialize the new column
df['clinic'] = pd.NA

# 1. Kidney Tx (renal or kidney)
mask_kidney = clinic_lower.str.contains('renal|kidney', na=False)
df.loc[mask_kidney, 'clinic'] = 'Kidney Tx'

# 2. Liver Tx (specific phrases)
mask_liver_specific = clinic_lower.str.contains(
    'pediatric liver transplant|adult pre liver transplant|adult post liver transplant',
    na=False
)
df.loc[mask_liver_specific & df['clinic'].isna(), 'clinic'] = 'liver Tx'

# 3. Anesthesia cases
mask_anesthesia = clinic_lower.str.contains('anesthesia', na=False)
df.loc[mask_anesthesia & df['clinic'].isna(), 'clinic'] = pd.NA  # default

df.loc[
    mask_anesthesia & df['clinic'].isna() & rd_lower.str.contains('kidney'),
    'clinic'
] = 'Anesthesia-Kidney'

df.loc[
    mask_anesthesia & df['clinic'].isna() & rd_lower.str.contains('liver'),
    'clinic'
] = 'Anesthesia-Liver'

# 4. Urology
mask_urology = clinic_lower.str.contains('transplant urology|transplant cystoscopy clinic', na=False)
df.loc[mask_urology & df['clinic'].isna(), 'clinic'] = 'Urology'

# 5. Nutrition
mask_nutrition = clinic_lower.str.contains('clinical nutrition', na=False)
df.loc[mask_nutrition & df['clinic'].isna(), 'clinic'] = 'Nutrition'

# 6. Surgery Dressing
mask_dressing = clinic_lower.str.contains(
    'transplant dressing (procedure)', 
    regex=False, 
    na=False
)
df.loc[mask_dressing & df['clinic'].isna(), 'clinic'] = 'Surgery-Dressing'

# 7. Transplant Surgery
mask_surgery = clinic_lower.str.contains('transplant surgery', na=False)
df.loc[mask_surgery & df['clinic'].isna(), 'clinic'] = 'Surgery-Liver'  # default for surgery

df.loc[
    mask_surgery & df['clinic'].isna() & rd_lower.str.contains('kidney'),
    'clinic'
] = 'Surgery-Kidney'

# 8. Psychology
mask_psych = clinic_lower.str.contains('psychology', na=False)
df.loc[mask_psych & df['clinic'].isna(), 'clinic'] = 'Psychology'

# 9. Social Work
mask_social = clinic_lower.str.contains('social work', na=False)
df.loc[mask_social & df['clinic'].isna(), 'clinic'] = 'Social Work'



In [70]:
df['clinic'].value_counts()

clinic
Kidney Tx            929
Surgery-Liver        410
liver Tx             338
Anesthesia-Kidney     82
Nutrition             76
Social Work           46
Surgery-Dressing      38
Anesthesia-Liver      24
Psychology            15
Urology                6
Name: count, dtype: int64

In [71]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days,age_group,clinic
1,187647,ALREEM AHMED ABDULKAREEM KHALAF,2016-08-31,2026-01-06,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,Mohammed Abdulmoatani Almaghrabi,NaN,NaN,Post LD Tx,NaN,no,Kidney Recipient,Post-Kidney Tx,2024-01-23,714.0,Pediatric,Kidney Tx
2,218085,Sultan Ali Saleh Alnahdi,1990-07-01,2026-01-13,NEW,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,ABDULNASER MOHAMMED HAMOUD ALABADI,NaN,NaN,NaN,Post living donor nephrectomy,no,Kidney Donor,Post-Kidney Tx,2025-12-02,42.0,Adult,Kidney Tx
4,147118,SAAD ABDULLAH ALI ALBUAINAIN,2008-12-31,2026-01-25,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,Recipient,Hepatology patient,Post DD Tx,NaN,no,Liver Recipient,Hepatology,2024-01-22,734.0,Pediatric,Kidney Tx
6,210109,YASAMIYAN HAMAD MOHAMMED ALRAWILI,2008-05-02,2026-01-20,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Kidney Recipient,Post-Kidney Tx,2025-09-15,127.0,Adult,Kidney Tx
28,176279,RADHI NASER HASAN ALHODAR,1946-05-31,2026-01-04,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Ebtisam Hassan Husain Al-Ghawi,NaN,NaN,Nephrology patient,NaN,no,Kidney Recipient,Nephrology,NaT,NaN,Adult,Nutrition


In [72]:
df['visit_type'].value_counts()

visit_type
FOLLOW UP            782
NEW                  482
V Follow UP          299
Walk-In              129
Kidney               126
CHART CHECK           77
Liver                 39
new                   14
VV                    12
Medication Refill      3
Follow up              1
Name: count, dtype: int64

In [73]:
df['clinic_name'].value_counts()

clinic_name
Adult Post Renal Transplant                        469
TRANSPLANT SURGERY                                 410
Adult Post Kidney Transplant Virtual Clinic        228
ADULT POST LIVER TRANSPLANT                        128
Anesthesia                                         102
Adult Pre Renal Transplant                          96
Adult Pre Liver Transplant                          88
Pediatric Post Renal Transplant                     85
Clinical Nutrition                                  76
Pediatric Liver Transplant                          72
Social Work                                         46
Adult Post Liver Transplant Virtual                 40
Transplant Dressing (procedure)                     38
Pediatric Post Kidney Transplant Virtual Clinic     21
Adult Pre Renal Transplant Virtual Clinic           19
Psychology                                          15
Pediatric Pre Renal Transplant                      11
Adult Pre Liver Transplant Virtual                  1

In [74]:
ref = pd.read_excel(MOTC_FILE)
ref.columns
ref.head(1)

########################

,Date,MRN,Name,Age,patient Type,clinic,Visit Type,R/D,Adult Kidney post type,Adult Kidney Tx out & donors,Pediatric Kidney Tx. Type,"Consultant Covered the clinic ""Actual Coverage""",Medica - Plus Clinic Name,Consultant name from Medica +,Assistant Coverage,In MCC,Unnamed: 16,Unnamed: 17,From papers
0,2018-01-07,89742,Fatima Ahmed Al-swaidani,Adult,Post-Liver tx,Liver Tx,FU,Liver,-,-,NaN,Mohammed Mawaldi,Adult Post Liver Tx,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
# =============================================================================
# COMPLETE OPD VISIT_TYPE CLEANING PIPELINE – FULL LOGIC EXPLAINED
# =============================================================================
#
# BUSINESS PROBLEM:
#   Our main dataframe (df) has a column 'visit_type' with messy/inconsistent values:
#   Examples: "Follow up", "New", "follow-up", "Kidney", "Liver", "Walk-In", "CHART CHECK", "VV", etc.
#   Goal: After cleaning, this column MUST contain ONLY these 4 allowed values:
#       → 'FOLLOW UP', 'NEW', 'V FOLLOW UP', 'VV'
#
# REFERENCE DATA:
#   We have an Excel file: "MOTC OPD visits - from 2018-2021.xlsx"
#   It contains historical visits with columns: 'MRN' and 'clinic'
#   We use it to decide whether a patient is known (FOLLOW UP) or new (NEW)
#
# STEP-BY-STEP LOGIC OF THE ENTIRE PIPELINE:
#
# 1. STANDARDIZE OBVIOUS CASES (simple fixes)
#    → Convert variations like "follow up", "Follow-up", "followup", "new" → canonical forms
#    → Make everything UPPERCASE and strip extra spaces
#    → Goal: Fix most clean cases without needing the reference file
#
# 2. IDENTIFY MESSY ROWS THAT STILL NEED HELP
#    → Create a mask: rows where visit_type is NOT in {'FOLLOW UP', 'NEW', 'V FOLLOW UP', 'VV'}
#    → These are the rows we couldn't automatically classify in step 1 (e.g. "Kidney", "Walk-In")
#
# 3. PREPARE REFERENCE DATA FOR FAST LOOKUPS
#    → Standardize 'clinic' and 'MRN' in the reference file (uppercase + strip)
#    → Create a set of known (clinic, MRN) pairs that have visited before
#
# 4. RECLASSIFY THE MESSY ROWS USING THE REFERENCE FILE (VECTORIZED – FAST!)
#    → For each messy row:
#        - Take its standardized clinic + MRN
#        - Check if this exact (clinic, MRN) pair exists in the historical reference data
#            → If YES → patient is known → set visit_type = 'FOLLOW UP'
#            → If NO  → patient is new   → set visit_type = 'NEW'
#    → We use pandas merge + np.where → very fast even for large datasets
#
# 5. FINAL SAFETY CHECK & SELECTIVE MANUAL REVIEW FLAGGING
#    → After all cleaning is done, check again:
#        - Print the final distribution of visit_type
#        - Look for ANY rows that still have invalid values (not in the 4 allowed ones)
#    → If there ARE still bad rows (should be rare!):
#        - Print a WARNING + show the unexpected values
#        - Flag ONLY these problematic rows for manual human review:
#            → Set need_review = 'yes' (column already exists and defaults to 'no')
#    → If everything is clean → print SUCCESS
#    → Important: We do NOT flag the 372 rows we successfully reclassified — only flag rows
#      that are STILL wrong after all automated logic
#
# WHY THIS APPROACH IS GREAT:
#   - Fast & scalable (vectorized merge instead of slow row-by-row loops)
#   - Auditable: we can trace every decision
#   - Safe: human review only for truly uncertain/problematic cases
#   - Future-proof: handles casing, spaces, missing values gracefully
#


# ── Inputs ────────────────────────────────────────────────────────
# REF_PATH = "../data/raw/MOTC OPD visits - from 2018-2021.xlsx"
ALLOWED_VISIT_TYPES = {"FOLLOW UP", "NEW", "V FOLLOW UP", "VV"}

# ── 1. Load reference ─────────────────────────────────────────────
ref = pd.read_excel(MOTC_FILE)

# ── 2. Standardize everything ─────────────────────────────────────
# visit_type
df["visit_type"] = df["visit_type"].astype(str).str.strip()
df.loc[
    df["visit_type"].str.lower().isin(["follow up", "follow-up", "followup"]),
    "visit_type"
] = "FOLLOW UP"
df.loc[df["visit_type"].str.lower() == "new", "visit_type"] = "NEW"
df["visit_type"] = df["visit_type"].str.upper().str.strip()

# clinic & mrn (helper columns)
df["clinic_std"] = df["clinic"].astype(str).str.strip().str.upper()
ref["clinic_std"] = ref["clinic"].astype(str).str.strip().str.upper()

df["mrn_std"] = df["mrn"].astype(str).str.strip()
ref["mrn_std"] = ref["MRN"].astype(str).str.strip()

# ── 3. Identify rows to fix ───────────────────────────────────────
needs_reclass = ~df["visit_type"].isin(ALLOWED_VISIT_TYPES)
print(f"Rows needing reclassification: {needs_reclass.sum():,}")

# ── 4. Vectorized reclassification (best way!) ────────────────────
ref_pairs = ref[["clinic_std", "mrn_std"]].drop_duplicates()

tmp = df.loc[needs_reclass, ["clinic_std", "mrn_std"]].merge(
    ref_pairs.assign(seen_in_ref=True),
    on=["clinic_std", "mrn_std"],
    how="left"
)

df.loc[needs_reclass, "visit_type"] = np.where(
    tmp["seen_in_ref"].notna(),   # ← .notna() returns True/False directly
    "FOLLOW UP",
    "NEW"
)

# ── 5. Cleanup ────────────────────────────────────────────────────
df.drop(columns=["clinic_std", "mrn_std"], inplace=True)

# ── Final safety check & flagging for review ───────────────────────
print("\nFinal visit_type distribution:")
print(df["visit_type"].value_counts(dropna=False))

# Find rows with unexpected (invalid) visit_type values
unexpected_mask = ~df["visit_type"].isin(ALLOWED_VISIT_TYPES)

# Get the actual unexpected values (for printing)
unexpected_values = df.loc[unexpected_mask, "visit_type"].unique()

if len(unexpected_values) > 0:
    print("\nWARNING — unexpected values still exist:", unexpected_values)
    print(f"Number of rows with unexpected values: {unexpected_mask.sum():,}")
    
    # Flag ONLY these problematic rows for manual review
    df.loc[unexpected_mask, 'need_review'] = 'yes'
    
    # Optional: also save what the bad value was
    if 'original_visit_type' not in df.columns:
        df['original_visit_type'] = df['visit_type'].copy()  # only if you didn't save it earlier
else:
    print("\nSUCCESS: All visit_types are now allowed! No rows need review.")

Rows needing reclassification: 374

Final visit_type distribution:
visit_type
FOLLOW UP      923
NEW            730
V FOLLOW UP    299
VV              12
Name: count, dtype: int64

SUCCESS: All visit_types are now allowed! No rows need review.


In [76]:
df['clinic'].value_counts()

clinic
Kidney Tx            929
Surgery-Liver        410
liver Tx             338
Anesthesia-Kidney     82
Nutrition             76
Social Work           46
Surgery-Dressing      38
Anesthesia-Liver      24
Psychology            15
Urology                6
Name: count, dtype: int64

In [77]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days,age_group,clinic
1,187647,ALREEM AHMED ABDULKAREEM KHALAF,2016-08-31,2026-01-06,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,Mohammed Abdulmoatani Almaghrabi,NaN,NaN,Post LD Tx,NaN,no,Kidney Recipient,Post-Kidney Tx,2024-01-23,714.0,Pediatric,Kidney Tx
2,218085,Sultan Ali Saleh Alnahdi,1990-07-01,2026-01-13,NEW,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,ABDULNASER MOHAMMED HAMOUD ALABADI,NaN,NaN,NaN,Post living donor nephrectomy,no,Kidney Donor,Post-Kidney Tx,2025-12-02,42.0,Adult,Kidney Tx
4,147118,SAAD ABDULLAH ALI ALBUAINAIN,2008-12-31,2026-01-25,FOLLOW UP,STARTED,NORMAL BOOKING,Pediatric Post Renal Transplant,ABDULAZIZ AHMED SALEM BA MHRAZ,Recipient,Hepatology patient,Post DD Tx,NaN,no,Liver Recipient,Hepatology,2024-01-22,734.0,Pediatric,Kidney Tx
6,210109,YASAMIYAN HAMAD MOHAMMED ALRAWILI,2008-05-02,2026-01-20,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Renal Transplant,SUMAYAH ABDULAZIZ MOHAMMED ASKANDARANI,NaN,NaN,Post LD Tx,NaN,no,Kidney Recipient,Post-Kidney Tx,2025-09-15,127.0,Adult,Kidney Tx
28,176279,RADHI NASER HASAN ALHODAR,1946-05-31,2026-01-04,NEW,STARTED,NORMAL BOOKING,Clinical Nutrition,Ebtisam Hassan Husain Al-Ghawi,NaN,NaN,Nephrology patient,NaN,no,Kidney Recipient,Nephrology,NaT,NaN,Adult,Nutrition


In [78]:
# =============================================================================
# FINAL STEP: FIX INCORRECT MULTIPLE "NEW" VISITS PER (MRN, CLINIC)
# =============================================================================
#
# BUSINESS RULE:
#   - Each patient (mrn) can have AT MOST ONE 'NEW' visit per clinic
#   - The 'NEW' must be the EARLIEST visit (by visit_datetime)
#   - All subsequent visits in the same clinic MUST be 'FOLLOW UP'
#
# We ONLY fix groups where:
#   - Multiple visits exist (group size > 1)
#   - AND ALL visits are currently labeled 'NEW'
#
# Groups we leave untouched:
#   - Single visit
#   - Already contain at least one 'FOLLOW UP', 'V FOLLOW UP' or 'VV'
#
# Important: We do NOT flag any rows for review in this step.
#            The need_review column remains unchanged.
#
# =============================================================================


# 1. Ensure visit_datetime is proper datetime
df['visit_datetime'] = pd.to_datetime(df['visit_datetime'], errors='coerce')

# 2. Sort chronologically within each (mrn, clinic) using stable sort
df = df.sort_values(
    ['mrn', 'clinic', 'visit_datetime'],
    kind='mergesort'                  # stable sort for reproducibility
).reset_index(drop=True)

# 3. Helper columns (all vectorized)
df['is_first_visit'] = df.groupby(['mrn', 'clinic']).cumcount() == 0
df['group_size'] = df.groupby(['mrn', 'clinic'])['visit_type'].transform('size')

# Safe & clear "all NEW" check
df['all_new'] = (
    df.groupby(['mrn', 'clinic'])['visit_type'].transform('nunique') == 1
) & df['visit_type'].eq('NEW')

# 4. Mask: rows we will change to 'FOLLOW UP'
mask_to_change = (
    (df['group_size'] > 1) &
    df['all_new'] &
    ~df['is_first_visit']
)

# 5. Store examples BEFORE mutation (shows original 'NEW' values)
changed_examples = df.loc[mask_to_change].copy()

# 6. Apply the correction
df.loc[mask_to_change, 'visit_type'] = 'FOLLOW UP'

# 7. Clean up helper columns
df = df.drop(columns=['is_first_visit', 'group_size', 'all_new'])



In [79]:
df.head()

,mrn,patengname,pat_birthdate,visit_datetime,visit_type,visit_status,booking_type,clinic_name,mrp,liver_r_d,liver_status,kidney_recipient,kidney_donor,need_review,r_d,patient_type,transplant_date,days,age_group,clinic
0,99,IBRAHIM AHMED YOUSEF ALAHSAIE,1956-02-13,2026-01-25,FOLLOW UP,STARTED,NORMAL BOOKING,ADULT POST LIVER TRANSPLANT,HADI MOHAMMED SHOEI KURIRY,Recipient,Post Tx,Nephrology patient,NaN,no,Liver Recipient,Post-Liver Tx,2007-05-31,6814.0,Adult,liver Tx
1,733,HUDA ABDULLAH AHMED ALDARORAH,1968-09-23,2026-01-11,V FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Liver Transplant Virtual,HADI MOHAMMED SHOEI KURIRY,Recipient,Post Tx,NaN,NaN,no,Liver Recipient,Post-Liver Tx,2018-10-28,2632.0,Adult,liver Tx
2,733,HUDA ABDULLAH AHMED ALDARORAH,1968-09-23,2026-01-18,V FOLLOW UP,STARTED,NORMAL BOOKING,Adult Post Liver Transplant Virtual,HADI MOHAMMED SHOEI KURIRY,Recipient,Post Tx,NaN,NaN,no,Liver Recipient,Post-Liver Tx,2018-10-28,2639.0,Adult,liver Tx
3,1470,KHALED SAEED BIN AWADH ALQAHTANI,1986-04-26,2026-01-06,NEW,STARTED,NORMAL BOOKING,Anesthesia,ASHRAF ALI MOUSSA ALI,Recipient,Hepatology patient,NaN,NaN,no,Liver Recipient,Hepatology,NaT,NaN,Adult,Anesthesia-Liver
4,1470,KHALED SAEED BIN AWADH ALQAHTANI,1986-04-26,2026-01-27,FOLLOW UP,STARTED,NORMAL BOOKING,Adult Pre Liver Transplant,NOORA HASSAN B AL FARAJ,Recipient,Hepatology patient,NaN,NaN,no,Liver Recipient,Hepatology,NaT,NaN,Adult,liver Tx


In [80]:
df.columns

Index(['mrn', 'patengname', 'pat_birthdate', 'visit_datetime', 'visit_type',
       'visit_status', 'booking_type', 'clinic_name', 'mrp', 'liver_r_d',
       'liver_status', 'kidney_recipient', 'kidney_donor', 'need_review',
       'r_d', 'patient_type', 'transplant_date', 'days', 'age_group',
       'clinic'],
      dtype='object')

In [81]:
df['clinic_name'].value_counts()

clinic_name
Adult Post Renal Transplant                        469
TRANSPLANT SURGERY                                 410
Adult Post Kidney Transplant Virtual Clinic        228
ADULT POST LIVER TRANSPLANT                        128
Anesthesia                                         102
Adult Pre Renal Transplant                          96
Adult Pre Liver Transplant                          88
Pediatric Post Renal Transplant                     85
Clinical Nutrition                                  76
Pediatric Liver Transplant                          72
Social Work                                         46
Adult Post Liver Transplant Virtual                 40
Transplant Dressing (procedure)                     38
Pediatric Post Kidney Transplant Virtual Clinic     21
Adult Pre Renal Transplant Virtual Clinic           19
Psychology                                          15
Pediatric Pre Renal Transplant                      11
Adult Pre Liver Transplant Virtual                  1

In [82]:
import os
from datetime import datetime


# -----------------------------
# 1) Build the final export_df
# -----------------------------
export_df = df.rename(columns={
    "visit_datetime": "date",
    "patengname": "name",
    "age_group": "age",
    "patient_type": "patient type",
    "visit_type": "visit type",
    "r_d": "r/d",
    "clinic_name": "Medica - Plus Clinic Name",
    "mrp": "Consultant name from Medica +",
})

final_cols = [
    "date",
    "mrn",
    "name",
    "age",
    "patient type",
    "clinic",
    "visit type",
    "r/d",
    "Medica - Plus Clinic Name",
    "Consultant name from Medica +",
    "need_review",
]
export_df = export_df[final_cols].copy()

# -----------------------------
# 2) Date format: DATE ONLY
#    (Excel will show it as a date)
# -----------------------------
export_df["date"] = pd.to_datetime(export_df["date"], errors="coerce").dt.date

# -----------------------------
# 3) Export to data/output as Excel
# -----------------------------

output_dir = "../data/output"
os.makedirs(output_dir, exist_ok=True)


year_month = datetime.now().strftime("%Y_%m")
output_path = os.path.join(
    output_dir,
    f"cleaned_opd_visits_{year_month}.xlsx"
)

export_df.to_excel(output_path, index=False)

print(f"✅ Exported: {output_path}")



✅ Exported: ../data/output/cleaned_opd_visits_2026_02.xlsx
